In [1]:
import pandas
import numpy
import priortisiation

# Inputs

In [2]:
cei_recc_dict = { "6.4.2" : "Scan ($object_type) weekly for vulnerabilties in ($attribute)" ,
"6.3.1":  "Scan ($object_type) daily for vulnerabilities in ($attribute)", 
"6.4.1" :  "Scan ($object_type) weekly for vulnerabilities in ($attribute)",
"6.6.1": "Upgrade operating systems of ($object_type) that are no longer supported by vendors in ($attribute)."
}

In [3]:
host_type = {"External Server":4, "Internal Server": 3, "Workstation": 3}
cei_code = {"6.4.2":1,"6.3.1":3,"6.4.1":2,"6.6.1":4}

# Input Data Preparation

In [4]:
cei_metric = pandas.read_parquet("Data/CEI Metric/")

In [5]:
cei_metric = cei_metric[cei_metric["analysis_period_day"] == cei_metric["analysis_period_day"].max()]

In [6]:
cei_metric

,host_id,host_name,os_version,source_of_truth,agent_name,agent_version,host_type,device_last_seen_date,antimalware_status,crowdstrike_compatible,...,analysis_period_week_start_epoch,analysis_period_week_end_epoch,object_type,row_uuid,elapsed_days,analysis_period_week_year,analysis_period_week,cei_code,product_name,location
1000,6931686f3977,i1ho9wzp9k,None,Qualys,None,None,Server,None,None,None,...,1650844800000,1651449599999,object_type,2740a86d-b347-41bb-9317-05e89b598cff-165136319...,11.0,2022,17,6.6.1,Product 01,India
1001,34656b703871,4ekp8q-h6cgl,None,Qualys,None,None,Server,None,None,None,...,1650844800000,1651449599999,object_type,d2ed0f24-abd1-463d-be90-a934d21ffacd-165136319...,11.0,2022,17,6.6.1,Product 01,Australia
1002,643073676f6e,d0sgonvi,None,Qualys,None,None,Server,None,None,None,...,1650844800000,1651449599999,object_type,9a41a149-0265-4dbb-a48a-53a3f8ec16d3-165136319...,11.0,2022,17,6.6.1,Product 01,Australia
1003,76626e6b3472,vbnk4rhnkn,None,Qualys,None,None,Server,None,None,None,...,1650844800000,1651449599999,object_type,ff05b394-de4e-4a80-8db8-97dd4df04962-165136319...,11.0,2022,17,6.6.1,Product 01,Australia
1004,367162743267,6qbt2gcz2,None,Qualys,None,None,Server,None,None,None,...,1650844800000,1651449599999,object_type,8570e33e-05e3-49a1-a2ce-8c4ceb84ba3b-165136319...,11.0,2022,17,6.6.1,Product 01,India
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8295,67376f6f2d64,g7oo-d5e,None,Qualys,None,None,Server,None,None,None,...,1650844800000,1651449599999,object_type,872d28bb-c4bb-4f3a-b50c-257681af2984-165136319...,1.0,2022,17,6.3.1,Product 03,United Kingdom
8296,6d686d716130,mhmqa0bm,None,Qualys,None,None,Server,None,None,None,...,1650844800000,1651449599999,object_type,d01e9030-ed8c-41b9-b92b-e814ad20289e-165136319...,1.0,2022,17,6.3.1,Product 01,Australia
8297,32376833792d,27h3y-v95g,None,Qualys,None,None,Server,None,None,None,...,1650844800000,1651449599999,object_type,90328fe7-e07a-444f-93a0-f44e0e9bf473-165136319...,1.0,2022,17,6.3.1,Product 02,United Kingdom
8298,6667612d676c,fga-gli,None,Qualys,None,None,Server,None,None,None,...,1650844800000,1651449599999,object_type,49cae67e-83fd-4f90-8043-60e9782c4bad-165136319...,1.0,2022,17,6.3.1,Product 01,Australia


In [7]:
cei_metric["object_type"] = cei_metric.fillna("-").apply(lambda row: row["accessibility"] + "_" + row["host_type"],axis=1).str.replace("-_","").str.replace("_"," ")
cei_metric["entity"] = "Host"

In [8]:
cei_data = cei_metric.fillna("-").groupby(["object_type","cei_code"])["cei_status"].value_counts(normalize=True)
cei_data = cei_data.to_frame()
cei_data.columns = ["failure_count"]
cei_data = cei_data.reset_index()
cei_failures = cei_data[cei_data["cei_status"] == "Failed"]
cei_failures = cei_failures.set_index("object_type")
cei_failures.drop("cei_status",axis=1,inplace=True)
cei_failures = cei_failures.pivot(columns="cei_code").fillna(0)
cei_failures.columns = cei_failures.columns.droplevel()

In [14]:
cei_data

,object_type,cei_code,cei_status,failure_count
0,External Server,6.3.1,Successful,0.900000
1,External Server,6.3.1,Failed,0.100000
2,External Server,6.4.2,Successful,0.950000
3,External Server,6.4.2,Failed,0.050000
4,External Server,6.6.1,Successful,1.000000
5,Internal Server,6.4.2,Successful,0.900000
6,Internal Server,6.4.2,Failed,0.100000
7,Internal Server,6.6.1,Successful,1.000000
8,Workstation,6.4.1,Successful,0.928571
9,Workstation,6.4.1,Failed,0.071429


In [10]:
cei_failures.index

Index(['External Server', 'Internal Server', 'Workstation'], dtype='object', name='object_type')

# Priortization and Insights

In [11]:
from datetime import datetime

In [12]:
start_time = datetime.now().timestamp()
priority_df = priortisiation.priority_rank(host_type,cei_code,cei_failures,"object_type","cei_code")
cei_metric_count = cei_metric.merge(priority_df,on=["object_type","cei_code"])
insights_df = priortisiation.insights(cei_metric_count,cei_recc_dict,"cei_status",["location","product_name"],["Location","Product"],"cei_code","object_type","entity","priority_rank","impact")
end_time = datetime.now().timestamp()


Iteration =  0  f(x) =  inf
Iteration =  1  f(x) =  0.5241737017012237
Iteration =  2  f(x) =  0.5241737017012237
Iteration =  3  f(x) =  0.5241737017012237
Iteration =  4  f(x) =  0.3426116801419893
Iteration =  5  f(x) =  0.29740645636117224
Iteration =  6  f(x) =  0.2848121489098157
Iteration =  7  f(x) =  0.2848121489098157
Iteration =  8  f(x) =  0.1531783261018581
Iteration =  9  f(x) =  0.1531783261018581
Iteration =  10  f(x) =  0.1531783261018581
Iteration =  11  f(x) =  0.1531783261018581
Iteration =  12  f(x) =  0.1531783261018581
Iteration =  13  f(x) =  0.1531783261018581
Iteration =  14  f(x) =  0.1531783261018581
Iteration =  15  f(x) =  0.1531783261018581
Iteration =  16  f(x) =  0.1531783261018581
Iteration =  17  f(x) =  0.1531783261018581
Iteration =  18  f(x) =  0.1531783261018581
Iteration =  19  f(x) =  0.1531783261018581
Iteration =  20  f(x) =  0.1531783261018581
Iteration =  21  f(x) =  0.1531783261018581
Iteration =  22  f(x) =  0.1531783261018581
Iteration = 

In [13]:
priority_df

,object_type,cei_code,priority_rank,priority_score
2,External Server,6.3.1,1.0,1.171857
4,Workstation,6.6.1,2.0,0.988940
5,External Server,6.4.2,3.0,0.971279
8,Internal Server,6.4.2,4.0,0.729498
10,Workstation,6.4.1,5.0,0.676419


In [ ]:
insights_df

In [ ]:
priority_df

In [ ]:
end_time - start_time

# Insights

In [ ]:
insights_df["cei_severity"] = insights_df["cei_code"].apply(lambda x: cei_code[x])
insights_df["object_severity"] = insights_df["object_type"].apply(lambda x: host_type[x])

In [ ]:
insights_df[["entity","cei_code","cei_severity","object_type","object_severity","priority_rank","attribute" ,"attribute_value","failure_count","total_scope","reccomendation"]]

In [ ]:
insights_df.to_csv("insights.csv",index=False)

In [ ]:
risk_metrix = priortisiation.effective_risk_matrix_calculation(cei_code,host_type,cei_failures)

In [ ]:
risk_metrix

In [ ]:
priortisiation.derive_cei_weights_bw(risk_metrix, cei_code)

In [ ]:
priortisiation.object_priority_rank(risk_metrix, [0.13898045, 0.20854346, 0.16610849, 0.4863676])

In [ ]:
object_type_column

In [ ]:
priority_rank = priortisiation.priority_rank(host_type, cei_code, cei_failures,"object_type","cei_code")

In [ ]:
insights_df

In [ ]:
priortisiation.priority_score_calculation([0.13898045, 0.20854346, 0.16610849, 0.4863676], priority_rank, risk_metrix, "object_type","cei_code")